#### Post process data for data viz 

In [1]:
import os,sys
sys.path.insert(0,'../../libs')
import pandas as pd

In [2]:
data_folder = '/data/chuang/Language_Model_Training_Data/Models/Topic_Models/step_10000'
#merged_out_path = os.path.join(data_folder,'topic_v2_merged_agg_data.csv')
merged_out_simple_path = os.path.join(data_folder,'topic_v2_merged_agg_data_simple.csv')


In [3]:
df = pd.read_csv(merged_out_simple_path)

In [4]:
df.columns

Index(['index', 'File_Name', 'Title', 'Country Code', 'Country_Name', 'Year',
       'income', 'REO Region', 'par', 'org_topic_id', 'prob_topic_id',
       'max_probability', 'non-program_topic_id',
       'non-program_max_probability', 'final_topic_id', 'CustomName',
       'level_0', 'level_1', 'level_2'],
      dtype='object')

#### 

pip install git+https://github.com/JosephBARBIERDARNAL/pypalettes.git
- https://python-graph-gallery.com/590-advanced-treemap/
- other [examples](https://python-graph-gallery.com/web-stacked-area-with-inflexion-arrows/)

In [5]:
# load libraries
import squarify # pip install squarify (algorithm for treemap)
import matplotlib.pyplot as plt
#from pypalettes import load_cmap
import pandas as pd
from highlight_text import fig_text
import numpy as np

In [6]:
collapsed_df = df.groupby(['level_0', 'level_1'], as_index=False)['File_Name'].count()
collapsed_df.head()

,level_0,level_1,File_Name
0,External,Balance Of Payments,16287
1,External,Commodity Market,3648
2,External,Exchange Rate Policies,3972
3,External,External Debt,4460
4,External,External Economic Environment,385


In [7]:
collapsed_df.head(100)

,level_0,level_1,File_Name
0,External,Balance Of Payments,16287
1,External,Commodity Market,3648
2,External,Exchange Rate Policies,3972
3,External,External Debt,4460
4,External,External Economic Environment,385
...,...,...,...
74,Structural/Social,Infrastructure,1494
75,Structural/Social,Labor Market,11427
76,Structural/Social,Social Welfare And Safety Nets,6992
77,Structural/Social,Structural Reform,7069


In [8]:
import plotly.express as px
import pandas as pd

# Create the treemap
fig = px.treemap(collapsed_df, 
                 path=['level_0', 'level_1'], 
                 values='File_Name',
                 #color='Name',
                 #color_continuous_scale='RdBu',
                 width=1000, 
                 height=800,
                 title="Treemap of AIV Topic Distribution")
fig.update_traces(hovertemplate='labels=%{label}<br>total_count=%{value}<extra></extra>')
# Show the plot
fig.show()

In [9]:
collapsed_df = df.groupby(['Year','level_0'], as_index=False)['File_Name'].count()
collapsed_df.head(10)

,Year,level_0,File_Name
0,2008,External,2458
1,2008,Financial,1837
2,2008,Fiscal,2844
3,2008,Monetary,633
4,2008,Others,1148
5,2008,Program,1057
6,2008,Real,1237
7,2008,Statistical Analysis,24
8,2008,Statistics,536
9,2008,Structural/Social,1598


In [10]:
fig = px.area(collapsed_df, 
              x="Year", 
              y="File_Name", 
              color="level_0", 
              line_group="level_0",
              height=500,
              width=1000,
              title='Topic Over Time',
              )
fig.update_layout(legend=dict(
                            title=dict(
                                text='High Level Topics',
                                font=dict(size=14)  # Optional: set the font size
                            ),),
                    yaxis_title='# of paragraphs'
    
                    )

fig.show()

In [11]:
collapsed_df['File_Name'] = collapsed_df.groupby('Year')['File_Name'].transform(lambda x: (x / x.sum()*100))
collapsed_df.head()

,Year,level_0,File_Name
0,2008,External,18.381693
1,2008,Financial,13.737661
2,2008,Fiscal,21.268322
3,2008,Monetary,4.733772
4,2008,Others,8.585103


In [12]:
fig = px.area(collapsed_df, 
              x="Year", 
              y="File_Name", 
              color="level_0", 
              line_group="level_0",
              height=500,
              width=1000,
              title='Topic Distribution Over Time',
              )
fig.update_layout(legend=dict(
                            title=dict(
                                text='High Level Topics',
                                font=dict(size=14)  # Optional: set the font size
                            ),),
                        yaxis=dict(
                                    title='Percentage',  # Optional: Set the y-axis title
                                    range=[0, 100]  # Set the y-axis range from 0 to 100
                                ),
    
                    )

fig.show()

In [13]:
collapsed_df = df.groupby(['Year','level_0','level_1'], as_index=False)['File_Name'].count()
collapsed_df = collapsed_df[collapsed_df['level_0']=='Structural/Social']
collapsed_df

,Year,level_0,level_1,File_Name
65,2008,Structural/Social,Business Environment,221
66,2008,Structural/Social,Climate,29
67,2008,Structural/Social,Financial Inclusion,7
68,2008,Structural/Social,Gender,49
69,2008,Structural/Social,Governance,162
...,...,...,...,...
1227,2023,Structural/Social,Infrastructure,110
1228,2023,Structural/Social,Labor Market,786
1229,2023,Structural/Social,Social Welfare And Safety Nets,536
1230,2023,Structural/Social,Structural Reform,509


In [14]:
fig = px.area(collapsed_df, 
              x="Year", 
              y="File_Name", 
              color="level_1", 
              line_group="level_1",
              height=500,
              width=1000,
              title='Structural/Social Topic Over Time',
              )
fig.update_layout(legend=dict(
                            title=dict(
                                text='High Level Topics',
                                font=dict(size=14)  # Optional: set the font size
                            ),),
                    yaxis_title='# of paragraphs'
    
                    )

In [15]:
collapsed_df['File_Name'] = collapsed_df.groupby('Year')['File_Name'].transform(lambda x: (x / x.sum()*100))


In [16]:
fig = px.area(collapsed_df, 
              x="Year", 
              y="File_Name", 
              color="level_1", 
              line_group="level_1",
              height=500,
              width=1000,
              title='Structural/Social Topic Distribution Over Time',
              )
fig.update_layout(legend=dict(
                            title=dict(
                                text='High Level Topics',
                                font=dict(size=14)  # Optional: set the font size
                            ),),
                        yaxis=dict(
                                    title='Percentage',  # Optional: Set the y-axis title
                                    range=[0, 100]  # Set the y-axis range from 0 to 100
                                ),
    
                    )

fig.show()